In [3]:
import * as E from "fp-ts/Either";
import * as A from "fp-ts/Array";
import * as F from "fp-ts/function";

import * as P from "./lib/promise";
import { request1, request2, request3, request4, request5, request6 } from './mocks/requests'


Simple sequential async requests

In [4]:
const sequential = request1(0)
    .then(request2)
    .then(request3)
    .then(request4)
    .then(request5)
    .then(request6);

console.log("sequential:", await sequential);

request1 arg: 0
request2 arg: 1
request3 arg: 2
request4 arg: 3
request5 arg: 4
request6 arg: 5
sequential: 6


Sometimes we need to keep the previous responses around, which causes nesting

In [5]:
const nested = request1(0).then(response1 =>
    request2({ response1 }).then(response2 =>
      request3({ response1, response2 }).then(response3 =>
        request4({ response1, response2, response3 }).then(response4 =>
          request5({ response1, response2, response3, response4 })
        )
      )
    )
  );

  console.log("nested:", await nested);

request1 arg: 0
request2 arg: { response1: 1 }
request3 arg: { response1: 1, response2: 2 }
request4 arg: { response1: 1, response2: 2, response3: 3 }
request5 arg: { response1: 1, response2: 2, response3: 3, response4: 4 }
nested: 5


Luckily, the `await`/`async` syntax helps with this

Although there are some issues: it hides the `Promise` data type away, it allows imperative style which can be tricky to follow (like awaiting within a loop or something)

Most importantly, `await`/`async` is specific to `Promises`, you cannot use it for other data types
 

In [6]:
const awaited1 = await request1(0);
const awaited2 = await request2({ awaited1 });
const awaited3 = await request3({ awaited1, awaited2 });
const awaited4 = await request4({ awaited1, awaited2, awaited3 });
const awaited5 = await request5({ awaited1, awaited2, awaited3, awaited4 });

console.log("awaited:", { awaited1, awaited2, awaited3, awaited4, awaited5 });

request1 arg: 0
request2 arg: { awaited1: 1 }
request3 arg: { awaited1: 1, awaited2: 2 }
request4 arg: { awaited1: 1, awaited2: 2, awaited3: 3 }
request5 arg: { awaited1: 1, awaited2: 2, awaited3: 3, awaited4: 4 }
awaited: { awaited1: 1, awaited2: 2, awaited3: 3, awaited4: 4, awaited5: 5 }


Enter `do notation`
this is equivalent to the previous `await`/`async` code

`bind` receives a key name and a function that returns a promise. The result of that function will be the value of the supplied key in an internal object
We can then access that internal object in each subsequent `bind`

In [7]:
const piped = F.pipe(
    P.Do,
    P.bind("response1", () => request1(0)),
    P.bind("response2", ({ response1 }) => request2({ response1 })),
    P.bind("response3", ({ response1, response2 }) => request3({ response1, response2 })),
    P.bind("response4", ({ response1, response2, response3 }) =>
      request4({ response1, response2, response3 })
    ),
    P.bind("response5", ({ response1, response2, response3, response4 }) =>
      request5({ response1, response2, response3, response4 })
    )
  );

  console.log("piped", await piped);

request1 arg: 0
request2 arg: { response1: 1 }
request3 arg: { response1: 1, response2: 2 }
request4 arg: { response1: 1, response2: 2, response3: 3 }
request5 arg: { response1: 1, response2: 2, response3: 3, response4: 4 }
piped {
  response1: 1,
  response2: 2,
  response3: 3,
  response4: 4,
  response5: 5
}


Sometimes, when the types are right, we can achieve more concise notation

In [8]:
const piped2 = F.pipe(
    P.Do,
    P.apS("response1", request1(0)),
    P.bind("response2", request2),
    P.bind("response3", request3),
    P.bind("response4", request4),
    P.bind("response5", request5)
  );
  
 console.log("piped", await piped2);

request1 arg: 0
request2 arg: { response1: 1 }
request3 arg: { response1: 1, response2: 2 }
request4 arg: { response1: 1, response2: 2, response3: 3 }
request5 arg: { response1: 1, response2: 2, response3: 3, response4: 4 }
piped {
  response1: 1,
  response2: 2,
  response3: 3,
  response4: 4,
  response5: 5
}
